In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
from mll_calc.mll_pred import format_XY

In [3]:
def logpdf_calc(row, test_sample, unc):
    y_sim = row[test_sample>0].values.tolist()
    std = row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

def loop_db(XY, pred, unc, lbls, nonlbls):
    logpdf_df = pd.DataFrame()
    for idx, row in pred.iterrows():
        sim_idx = row['sim_idx']
        pred_idx = row['pred_idx']
        all_lbls = lbls + nonlbls
        
        test_sample = XY.loc[sim_idx].drop(all_lbls)
        train_row = XY.loc[pred_idx].drop(all_lbls)
        
        logpdf = logpdf_calc(train_row, test_sample, unc)
        mll = row['MaxLogLL']
        
        if round(mll, 4) != round(np.sum(logpdf), 4):
            print('mismatch')
            break
        
        logpdf = pd.Series(logpdf, index=train_row[test_sample>0].index, name=sim_idx)
        if logpdf_df.empty:
            logpdf_df = pd.DataFrame(columns = test_sample.index.to_list())
        logpdf_df = logpdf_df.append(logpdf)
    return logpdf_df

In [4]:
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
nonlbls = ['AvgPowerDensity', 'ModDensity', 'UiWeight']

train15_pkl = '~/sims_n_results/simupdates_aug2020/not-scaled_nuc15.pkl'
XY15 = format_XY(train15_pkl)

train29_pkl = '~/sims_n_results/simupdates_aug2020/not-scaled_nuc15.pkl'
XY29 = format_XY(train29_pkl)

In [5]:
results15 = '~/sims_n_results/simupdates_aug2020/train15/'
results29 = '~/sims_n_results/simupdates_aug2020/train15/'

uncs = [0.05, ]#0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
dfs = {}
for i, unc_job in enumerate(job_dirs):
    preds15 = pd.read_csv(results15 + unc_job + '/' + unc_job + '.csv')
    preds29 = pd.read_csv(results29 + unc_job + '/' + unc_job + '.csv')

In [6]:
# overlapping predictions in both sets
to_print = ['sim_idx', 'pred_idx', 'AvgPowerDensity', 
            'ReactorType', 'pred_ReactorType', 'ReactorType_Score', 
            'Enrichment', 'pred_Enrichment', 'Enrichment_Error', 
            'Burnup', 'pred_Burnup', 'Burnup_Error',
            'CoolingTime', 'pred_CoolingTime', 'CoolingTime_Error',
            'OrigenReactor', 'pred_OrigenReactor', 'MaxLogLL']
preds15 = preds15.loc[:, to_print]
preds29 = preds29.loc[:, to_print]

preds = preds15.join(preds29, on = 'sim_idx', how = 'inner', lsuffix = '_l', rsuffix = '_r')

In [8]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [9]:
preds.sample(20)

,sim_idx,sim_idx_l,pred_idx_l,AvgPowerDensity_l,ReactorType_l,pred_ReactorType_l,ReactorType_Score_l,Enrichment_l,pred_Enrichment_l,Enrichment_Error_l,Burnup_l,pred_Burnup_l,Burnup_Error_l,CoolingTime_l,pred_CoolingTime_l,CoolingTime_Error_l,OrigenReactor_l,pred_OrigenReactor_l,MaxLogLL_l,sim_idx_r,pred_idx_r,AvgPowerDensity_r,ReactorType_r,pred_ReactorType_r,ReactorType_Score_r,Enrichment_r,pred_Enrichment_r,Enrichment_Error_r,Burnup_r,pred_Burnup_r,Burnup_Error_r,CoolingTime_r,pred_CoolingTime_r,CoolingTime_Error_r,OrigenReactor_r,pred_OrigenReactor_r,MaxLogLL_r
24,1029,1029,1028,10.0,bwr,bwr,True,0.5,0.50,0.00,30660.96,30660.96,0.00,473.850856,471.388214,2.462642,ge7x7-0,ge7x7-0,-50.838399,39417,220856,25.0,bwr,bwr,True,2.740,2.730,0.01,17668.74,18096.51,427.77,6160.818841,6159.232422,1.586419,ge7x7-0,svea64-1,-35.197104
177,6732,6732,6733,20.0,bwr,bwr,True,0.5,0.50,0.00,965.63,965.63,0.00,1032.998451,1046.132758,13.134307,ge7x7-0,ge7x7-0,9.162696,257530,257531,25.0,pwr,pwr,True,3.760,3.760,0.00,23261.53,23261.53,0.00,750.085316,848.060234,97.974918,ce14x14,ce14x14,-47.250258
43,1584,1584,1585,10.0,bwr,bwr,True,0.5,0.50,0.00,63534.57,63534.57,0.00,2074.891758,2079.301841,4.410083,ge7x7-0,ge7x7-0,-56.287381,60724,60723,10.0,bwr,bwr,True,0.550,0.550,0.00,5533.40,5533.40,0.00,27.412098,7.797336,19.614762,abb8x8-1,abb8x8-1,-26.370728
168,6435,6435,6375,10.0,bwr,bwr,True,0.5,0.50,0.00,48949.30,48334.78,614.52,1206.810070,1206.810070,0.000000,ge7x7-0,ge7x7-0,-54.629971,246110,246170,25.0,pwr,pwr,True,1.520,1.520,0.00,19111.96,19775.13,663.17,5333.959300,5333.959300,0.000000,ce14x14,ce14x14,-43.313419
247,9448,9448,9449,25.0,bwr,bwr,True,0.5,0.50,0.00,30660.96,30660.96,0.00,2598.806600,2625.070937,26.264337,ge7x7-0,ge7x7-0,-47.992152,361436,361437,22.0,phwr,phwr,True,0.711,0.711,0.00,7263.40,7263.40,0.00,6483.944237,6678.915522,194.971285,candu19,candu19,-27.004577
78,3053,3053,3054,20.0,bwr,bwr,True,0.5,0.50,0.00,48334.78,48334.78,0.00,5128.931533,5158.023377,29.091844,ge7x7-0,ge7x7-0,-50.135744,116814,116815,10.0,bwr,bwr,True,5.230,5.230,0.00,21800.82,21800.82,0.00,6026.246281,6103.568798,77.322517,abb8x8-1,abb8x8-1,-35.866421
197,7576,7576,7577,20.0,bwr,bwr,True,0.5,0.50,0.00,21745.58,21745.58,0.00,1250.512881,1272.233919,21.721038,ge7x7-0,ge7x7-0,-45.368293,289817,289816,41.0,pwr,pwr,True,0.500,0.500,0.00,1967.36,1967.36,0.00,1397.952960,1351.325953,46.627007,s18x18,s18x18,-6.608460
189,7299,7299,67779,20.0,bwr,bwr,True,0.5,0.55,0.05,13478.85,13558.75,79.90,3828.695214,3869.273243,40.578029,ge7x7-0,abb8x8-1,-36.641836,279128,279068,41.0,pwr,pwr,True,2.920,2.920,0.00,30039.95,29851.60,188.35,455.237804,455.237804,0.000000,w17x17,w17x17,-50.992094
128,4946,4946,4947,25.0,bwr,bwr,True,0.5,0.50,0.00,63534.57,63534.57,0.00,2311.579113,2355.494784,43.915671,ge7x7-0,ge7x7-0,-56.980939,189180,189239,20.0,bwr,bwr,True,0.500,0.500,0.00,30045.06,30045.06,0.00,0.000619,0.000000,0.000619,svea64-1,svea64-1,-50.090386
159,5979,5979,187362,10.0,bwr,bwr,True,0.5,0.50,0.00,22690.61,22475.28,215.33,3828.695214,3758.522130,70.173084,ge7x7-0,svea64-1,-44.299913,228610,228609,20.0,bwr,bwr,True,4.070,4.070,0.00,2796.41,2796.41,0.00,565.158549,523.199045,41.959504,svea64-1,svea64-1,-0.112459


In [13]:
to_print_2 = ['MaxLogLL_l', 'MaxLogLL_r', 'Burnup_Error_l', 'Burnup_Error_r', 
              'Enrichment_Error_l', 'Enrichment_Error_r', 
              'CoolingTime_Error_r', 'CoolingTime_Error_r']
preds.loc[:, to_print_2].sample(20)

,MaxLogLL_l,MaxLogLL_r,Burnup_Error_l,Burnup_Error_r,Enrichment_Error_l,Enrichment_Error_r,CoolingTime_Error_r,CoolingTime_Error_r
31,-46.267675,-44.812435,0.0,0.00,0.0,0.0,53.146807,53.146807
152,-44.104600,-35.870123,0.0,0.00,0.0,0.0,3.454678,3.454678
155,-45.248006,-46.236946,0.0,0.00,0.0,0.0,30.859317,30.859317
169,-52.039704,-52.976269,0.0,0.00,0.0,0.0,38.884825,38.884825
5,-27.242643,-54.171732,0.0,0.00,0.0,0.0,15.938932,15.938932
248,-50.738206,-28.710187,0.0,0.00,0.0,0.0,40.605195,40.605195
37,-55.499709,-26.323204,0.0,0.00,0.0,0.0,21.721038,21.721038
81,-52.069627,-14.127972,0.0,0.00,0.0,0.0,49.744153,49.744153
116,-46.324208,-43.718984,0.0,0.00,0.0,0.0,50.938446,50.938446
40,-54.682785,-59.654566,0.0,0.00,0.0,0.0,0.322040,0.322040


In [10]:
#preds15
preds['MaxLogLL_l'].describe()

count    251.000000
mean     -38.513872
std       16.269817
min      -60.082480
25%      -50.470185
50%      -42.287828
75%      -31.687015
max       13.742486
Name: MaxLogLL_l, dtype: float64

In [11]:
#preds29
preds['MaxLogLL_r'].describe()

count    251.000000
mean     -38.129809
std       18.475049
min      -62.220978
25%      -51.340667
50%      -43.174071
75%      -30.749305
max       26.853743
Name: MaxLogLL_r, dtype: float64

In [ ]:
unc = 0.05
logpdf15 = loop_db(XY15, preds15, unc, lbls, nonlbls)
logpdf29 = loop_db(XY29, preds29, unc, lbls, nonlbls)